In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras import layers as L
from tensorflow.keras import metrics as M
from keras.callbacks import LambdaCallback

In [ ]:
def create_model(emb_cols, emb_shape, num_shape):
    embed_out_dim = 1

    input_layers = []
    embeddings = []
    for emb_col in emb_cols:
        emb_size = emb_shape[emb_col]
        
        # Define Embedding Input Layer
        emb_input = Input(shape=(1,), name=f"{emb_col}_emb", dtype=tf.int32)
        
        # Define Embedding
        emb = L.Embedding(emb_size, embed_out_dim, name=emb_col)(emb_input)
        emb = L.Flatten()(emb)
        
        # Store in Dict
        input_layers.append(emb_input)
        embeddings.append(emb)
    
    # Numerical Feats Input
    num_inp = Input(shape=(num_shape,), name="num_feats", dtype=tf.float32)
    #bn = L.BatchNormalization( name='Normalize')(num_inp)
    num_emb = L.Dense(30, activation="relu", name="num_emb")(num_inp)
    
    base_embedding = tf.concat(embeddings + [num_emb], axis=1)
    dense1 = L.Dense(30, activation="relu", name="l1")(base_embedding)
    # dropout = L.Dropout(0.2)(dense1)
    dense2 = L.Dense(16, activation="relu", name="l2")(dense1)
    output = L.Dense(1, activation="sigmoid", name="output")(dense2)
    
    # Define Model
    model = Model(input_layers+[num_inp], output)

    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam',
                  metrics=[
                      M.AUC(name='auc'),
                      M.Precision(name='precision'),
                      M.Recall(name='recall')
                  ])
    
    print(model.summary())
    return model

In [ ]:
num_col_size = len(numerical_columns)

In [ ]:
model = create_model(categorical_columns, embedding_size_dict, num_col_size)
model

In [ ]:
training_history = model.fit(
    train_X, 
    train_df.is_fraud, 
    epochs=15, 
    batch_size=8192,
    verbose=1,
    validation_data=(test_X, test_df.is_fraud),
    class_weight = class_weights,
    #callbacks = [log_weights],
    #sample_weight = train_df.txns/train_df.txns.max()
)